In [22]:
import numpy as np
import cv2
import time
import pyautogui
from collections import defaultdict

In [23]:
def segment_by_angle_kmeans(lines, k=2, **kwargs):
    """Groups lines based on angle with k-means.

    Uses k-means on the coordinates of the angle on the unit circle 
    to segment `k` angles inside `lines`.
    """

    # Define criteria = (type, max_iter, epsilon)
    default_criteria_type = cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER
    criteria = kwargs.get('criteria', (default_criteria_type, 10, 1.0))
    flags = kwargs.get('flags', cv2.KMEANS_RANDOM_CENTERS)
    attempts = kwargs.get('attempts', 10)

    # returns angles in [0, pi] in radians
    angles = np.array([line[0][1] for line in lines])
    # multiply the angles by two and find coordinates of that angle
    pts = np.array([[np.cos(2*angle), np.sin(2*angle)]
                    for angle in angles], dtype=np.float32)

    # run kmeans on the coords
    labels, centers = cv2.kmeans(pts, k, None, criteria, attempts, flags)[1:]
    labels = labels.reshape(-1)  # transpose to row vec

    # segment lines based on their kmeans label
    segmented = defaultdict(list)
    for i, line in zip(range(len(lines)), lines):
        segmented[labels[i]].append(line)
    segmented = list(segmented.values())
    return segmented



In [24]:
def intersection(line1, line2):
    """Finds the intersection of two lines given in Hesse normal form.

    Returns closest integer pixel locations.
    """
    rho1, theta1 = line1[0]
    rho2, theta2 = line2[0]
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [[x0, y0]]




In [25]:
def segmented_intersections(lines):
    """Finds the intersections between groups of lines."""

    intersections = []
    for i, group in enumerate(lines[:-1]):
        for next_group in lines[i+1:]:
            for line1 in group:
                for line2 in next_group:
                    intersections.append(intersection(line1, line2)) 

    return intersections



In [26]:
img = cv2.imread('image.png')
img_p = img.copy()
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lr = np.array([0, 0, 0])
ur = np.array([255, 255, 70])

lrbl = np.array([0, 0, 0])
urbl = np.array([255, 255, 70])

lrb = np.array([102, 23, 113])
urb = np.array([137, 246, 257])

mask = cv2.inRange(hsv, lr, ur)

_, binary = cv2.threshold(mask, 0, 255, cv2.THRESH_OTSU)
edges = cv2.Canny(binary, threshold1=50, threshold2=200)

rho, theta, thresh = 2, np.pi/180, 475
lines = cv2.HoughLines(edges, rho, theta, thresh)

for line in lines:
    for rho,theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 3000*(-b))
        y1 = int(y0 + 3000*(a))
        x2 = int(x0 - 3000*(-b))
        y2 = int(y0 - 3000*(a))
        cv2.line(img_p,(x1,y1),(x2,y2),(0,255,0),2)

segmented = segment_by_angle_kmeans(lines)
intersections = segmented_intersections(segmented)

intersections.sort()

post_intcn = []
lastX, lastY = intersections[0][0]
for i in range(len(intersections)):
    x_1, y_1 = intersections[i][0]
    if (np.abs(lastX - x_1) > 0 and np.abs(lastX - x_1) < 15) or (np.abs(lastY - y_1) > 0 and np.abs(lastY - y_1) < 15):
        intersections[i][0] = 0, 0
    else:
        cv2.line(img_p, (x_1, y_1), (x_1, y_1), (255, 0, 0), 5)
        post_intcn.append([[x_1, y_1]])
    if np.abs(lastX - x_1) > 100:
        lastX = x_1
    if np.abs(lastY - y_1) > 25:
        lastY = y_1

lastX, lastY = post_intcn[0][0]
for i in range(len(post_intcn)):
    x_1, y_1 = post_intcn[i][0]
    if np.abs(lastX - x_1) < 15:
        post_intcn.append([[1, y_1]])

post_intcn.sort()
for i in range(len(post_intcn)):
    x_1, y_1 = post_intcn[i][0]

xmap = []
lastX = 0
for i in range(len(post_intcn)):
    x_1, y_1 = post_intcn[i][0]
    if np.abs(x_1 - lastX) > 0:
        xmap.append(x_1)
        lastX = x_1
xmap.append(xmap[len(xmap) - 1] + 1000)

for i in range(len(xmap)):
    x_1 = xmap[i]

k = 1
lastX, lastY = post_intcn[0][0]


In [27]:
for i in range(len(post_intcn) - 1):
    x_1, y_1 = post_intcn[i][0]
    x_2, y_2 = post_intcn[i + 1][0]
    x = xmap[k]
    if np.abs(x_1 - lastX) > 0 and k + 1 <= len(xmap) - 1:
        k = k + 1
        lastX = x_1
        x = xmap[k]
    if y_2 > y_1:
        if x - x_1 > 25 and y_2 - y_1 > 15:
            crop = img[y_1:y_1 + y_2 - y_1 + 3, x_1:x_1 + x - x_1]
            crop_hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
            trs = cv2.inRange(crop_hsv, lrb, urb)
            moments = cv2.moments(trs, 1)
            dArea = moments['m00']
            if dArea > 100:
                mask2 = cv2.inRange(crop_hsv, lrbl, urbl)

                src1_mask=cv2.cvtColor(mask2,cv2.COLOR_GRAY2BGR)
                res = cv2.bitwise_and(crop, crop, mask = trs)
                black_pixels = np.where(
                    (res[:, :, 0] == 0) & 
                    (res[:, :, 1] == 0) & 
                    (res[:, :, 2] == 0)
                )

                res[black_pixels] = [255, 255, 255]
                corr = False
                rows,cols,_ = res.shape
                for j in range(cols):
                    r = res[-1,j]
                    if np.any(r != 255):
                        corr = True

                cv2.line(img_p,(x_1,y_1),(x_1,y_1),(0,0,255),5)
                res = cv2.pyrMeanShiftFiltering(res, 10, 50)
                res = cv2.medianBlur(res, 3)

                if corr:
                    cv2.imwrite('screenshotCorrupted' + str(i) + '.png', res)
                else:
                    cv2.imwrite('screenshot' + str(i) + '.png', res)
        

In [28]:
cv2.imshow("Hough_line_p", img_p)
cv2.imwrite('copy.png', img_p)

cv2.waitKey(0)
cv2.destroyAllWindows()